In [7]:

from haystack.components.generators import GPTGenerator
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack import Pipeline
from haystack.components.retrievers import InMemoryBM25Retriever
from haystack.dataclasses import Document
from haystack.document_stores import InMemoryDocumentStore
from haystack.components.converters import PyPDFToDocument
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.writers import DocumentWriter
from pathlib import Path
from dotenv import load_dotenv
import os

load_dotenv("../../.env")

openai_api_key = os.getenv("OPENAI_API_KEY")


### Indexing pipeline 

In [8]:
document_store = InMemoryDocumentStore()

pipeline = Pipeline()
pipeline.add_component("converter", PyPDFToDocument())
pipeline.add_component("cleaner", DocumentCleaner())
pipeline.add_component("splitter", DocumentSplitter(split_by="sentence", split_length=5))
pipeline.add_component("writer", DocumentWriter(document_store=document_store))
pipeline.connect("converter", "cleaner")
pipeline.connect("cleaner", "splitter")
pipeline.connect("splitter", "writer")

pipeline.run({"converter": {"sources": [Path("./BJJMM_1_-_Mechanical_Models.pdf")]}})

{'writer': {'documents_written': 124}}

### RAG pipeline

In [10]:
llm =  GPTGenerator(api_key = openai_api_key, model_name = "gpt-4")

prompt_template = """
According to these documents:

{% for doc in documents %}
  {{ doc.content }}
{% endfor %}

Answer the given question: {{question}}
Answer:
"""


prompt_builder = PromptBuilder(template=prompt_template)
retriever = InMemoryBM25Retriever(document_store = document_store)


In [11]:
basic_rag_pipeline = Pipeline()

basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", llm)


basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")

basic_rag_pipeline.draw("../../images/mechanical_models_rag_pdf.png")

In [15]:
question = "Tell me about the mechanical models of the BJJ"

response = basic_rag_pipeline.run({"retriever": {"query": question}, 
                                   "prompt_builder": {"question": question}})


Ranking by BM25...:   0%|          | 0/124 [00:00<?, ? docs/s]

Ranking by BM25...: 100%|██████████| 124/124 [00:00<00:00, 22270.00 docs/s]


In [18]:
response['llm']['replies'][0].split("\n")

['The mechanical models of Brazilian Jiu-Jitsu (BJJ) as mentioned in the documents by Steve and Matt Kwan are categorized into several sections.',
 '',
 "1. Theory of Alignment which is about preserving your posture, structure, and base while attempting to disrupt your opponent's. Alignment serves as the framework for all mechanical aspects of BJJ.",
 '',
 '2. Core Mechanics which are a set of basic principles that dictate the fundamentals and structure of the technique in Brazilian Jiu-Jitsu.',
 '',
 '3. Anatomic Hierarchy which focuses on controlling and attacking the limbs and joints for optimum advantage.',
 '',
 '4. Different types of Guard, which can be classified as hook-based, clamp-based, frame-based, or hybrid to maintain a defensive position against the opponent.',
 '',
 '5. Mental Strategy for Recovery: acknowledging the role of mental recovery alongside physical training.',
 '',
 '6. Connection Correlation, Force Vectors, and Leading Edges for movement and control and unde

In [19]:
question = "Tell me the most important aspects of theory of alignment, how can I use it in my BJJ?"

response = basic_rag_pipeline.run({"retriever": {"query": question}, 
                                   "prompt_builder": {"question": question}})


Ranking by BM25...: 100%|██████████| 124/124 [00:00<00:00, 16702.86 docs/s]


In [20]:
response['llm']['replies'][0].split("\n")

['The Theory of Alignment in Brazilian Jiu-Jitsu (BJJ) refers to the ability to maintain your own posture, structure, and base, while trying to disrupt these elements in your opponent. ',
 '',
 '1. Posture: It refers to the effective positioning of your neck, core, and spine. Good posture in BJJ means creating a barrier or defense with your body positioning that makes it difficult for your opponent to attack you. For example, pulling down on your opponent’s head can break their posture.',
 '',
 "2. Structure: It's about efficient use of your limbs. Each limb's positioning can allow you to either maintain your own structure or disrupt your opponent's. Utilizing the arms and legs as levers can primarily attack your opponent's structure.",
 '',
 '3. Base: This is your ability to generate and absorb force relative to your goals. It refers to the way you distribute your weight and how you are connected to the ground. A good base can help you maintain balance and generate power for your atta

In [21]:
question = "Suggest BJJ positions that enable me to use the theory of alignment"

response = basic_rag_pipeline.run({"retriever": {"query": question}, 
                                   "prompt_builder": {"question": question}})


Ranking by BM25...: 100%|██████████| 124/124 [00:00<00:00, 16309.50 docs/s]


In [22]:
response['llm']['replies'][0].split("\n")

['The documents do not provide specific Brazilian Jiu-Jitsu (BJJ) positions that enable the use of the theory of alignment.']